链

链是LangChain的核心。它们是一系列组件，按特定顺序执行。

## 一个最简单的例子: prompt + model + output parser
目前官网推荐使用 LCEL 方式来构建 Chain，即 LangChain Expression Language。

需要配置 openai key

In [28]:
%env OPENAI_API_KEY=you-api-key

env: OPENAI_API_KEY=you-api-key


如果无法访问 openai api，可以配置 base url

In [29]:
%env OPENAI_API_BASE=you-base-url

env: OPENAI_API_BASE=you-base-url


我们可以额外定义一个函数去计算每次使用的 tokens 数量，这样我们就可以方便地查看每次调用 api 所消耗的 tokens

In [15]:
def chain_invoke_tokens(chain, prompt):
    with get_openai_callback() as cb:
        result = chain.invoke(prompt)
        print(f'Spent a total of {cb.total_tokens} tokens\n')

    return result

使用聊天模板，gpt3.5 turbo 模型，构建一个简单的 llm 链。

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(
    chain_invoke_tokens(chain, {"topic": "ice cream"})
)

Spent a total of 39 tokens
Why did the ice cream go to therapy? 

Because it had too many toppings and couldn't get its scoop together!


为了深入了解LangChain应用程序的内部工作，可以使用 set_verbose 和 set_debug 来了解引擎执行的细节。

In [26]:
from langchain.globals import set_verbose, set_debug

set_verbose(True)
set_debug(False)

加入 PythonREPLTool 工具，用于数学计算，构建新的链。

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain_experimental.tools import PythonREPLTool

prompt = ChatPromptTemplate.from_template("tell me a result about {query}")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()

python_repl = PythonREPLTool() 

chain = prompt | model | output_parser | python_repl

print(
    chain_invoke_tokens(chain, {"query": "What is 1 plus 1?"})
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "What is 1 plus 1?"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "query": "What is 1 plus 1?"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "tell me a result about What is 1 plus 1?",
          "additional_kwargs": {}
        }
      }
    ]
  }
}
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a result about What is 1 plus 1?"
  ]
}
[llm/end] [1:c